## Usage
- Edit the settings in the cell below.
- Cell -> Run All.
- A Napari window will open, where you can scroll through your data.

In [ ]:

# change this to point to your plate as seen in lmu_active1/instruments/Nano
plate = 'lyu/140324-A53T P62 staining/140324-A53T P62 staining/2024-03-27/20378/TimePoint_1'
plate = 'karkkael/microwell images/Plate1/2024-09-23/1/TimePoint_1/'

# set lmu_active1 root folder for Linux or Windows
#lmu_active1 = Path('/mnt/lmu_active1') # Linux
#lmu_active1 = Path('L:\lmu_active1') # Windows

# define colors you want to use (as many as you have channels)
#colormap = ["yellow", "magenta", "cyan"]
colormap = ["blue", "green", "red"]
colormap = ["blue", "green", "red", "magenta"]


## Code
You don't need to make changes in the code cells below. 

In [ ]:
from aicsimageio.aics_image import AICSImage
from pathlib import Path
import matplotlib.pyplot as plt
import napari
import numpy as np
import os
import pandas as pd
import platform

def get_lmu_active1():
    current_os = platform.system()
    
    if current_os == "Windows":
        return "L:\\lmu_active1"
    elif current_os == "Linux":
        return "/mnt/lmu_active1"
    else:
        raise ValueError(f"Unsupported operating system: {current_os}")
        
# original image folder
orig = get_lmu_active1() / Path('instruments/Nano') / Path(plate)
orig = Path('/home/user/nanodata') / Path(plate)
orig = Path('/home/hajaalin/data/micro') / Path(plate)


In [ ]:
files = [(str(x),x.parent,x.name) for x in orig.glob("**/*.tif") if not "thumb" in x.name]
df = pd.DataFrame(files, columns=['Path','DirName','FileName'])

files = [(str(x)) for x in orig.glob("**/*.tif") if not "thumb" in x.name]
#files = ['/home/user/data/ael/microwell images/Plate1/2024-09-23/1/TimePoint_1/ZStep_9/Plate1_E05_s2_w1D10A7B39-96B7-4967-8EC6-22EA232A7725.tif']
df = pd.DataFrame(files, columns=['Path'])

print(files[-1])
df.head()

DATE = 'Date'
TIMEPOINT = 'TimePoint'
ZSTEP = 'ZStep'
PLATE = 'Plate'
WELL = 'Well'
SITE = 'Site'
CHANNEL = 'Channel'

metadata_columns = {
    'mc1': DATE,
    'mc2': TIMEPOINT,
    'mc3': ZSTEP,
    'mc4': PLATE,
    'mc5': WELL,
    'mc6': SITE,
    'mc7': CHANNEL
}
# Regular expression pattern
pattern = r'.*/(?P<Date>\d\d\d\d-\d\d-\d\d)/[^/]*/TimePoint_(?P<TimePoint>\d+)/(?:ZStep_(?P<ZStep>\d+)/)?(?P<Plate>[^_]*)_(?P<Well>\w\d{2})_s(?P<Site>\d)_(?P<Wavelength>w\d)'

# Cross-platform pattern with dynamic column names
pattern = r'.*[/\\](?P<{mc1}>\d{{4}}-\d{{2}}-\d{{2}})[/\\][^/\\]*[/\\]TimePoint_(?P<{mc2}>\d+)(?:[/\\]ZStep_(?P<{mc3}>\d+))?[/\\](?P<{mc4}>[^_]+)_(?P<{mc5}>\w\d{{2}})_s(?P<{mc6}>\d)_(?P<{mc7}>w\d)'.format(**metadata_columns)

# Apply the regex pattern and extract the desired columns
df_extracted = df['Path'].str.extract(pattern)
print()

# Add the extracted columns back to the original dataframe
df = df.join(df_extracted)

# Show the result
df.head()

In [ ]:
df.tail()

In [ ]:
files[-1]

In [ ]:
wavelengths = sorted(df.Channel.unique())
wavelengths

In [ ]:
metadata_cols = metadata_columns.values()
metadata_cols

In [ ]:
mask = df[ZSTEP].isnull()
df2d = df[mask].copy().reset_index(drop=True)
df3d = df[~mask].copy().reset_index(drop=True)
df3d[ZSTEP] = df3d[ZSTEP].astype(int)

df2d.sort_values(by=[PLATE, WELL, SITE, CHANNEL], inplace=True, ignore_index=True)
df3d.sort_values(by=[PLATE, WELL, SITE, ZSTEP, CHANNEL], inplace=True, ignore_index=True)
df3d.head()

In [ ]:
grouped2d = df2d.groupby(by=[PLATE, WELL, SITE]).agg(list)
grouped2d

In [ ]:
grouped3d = df3d.groupby(by=[PLATE, WELL, SITE]).agg(list)
grouped3d

In [ ]:
import dask.array as da
import numpy as np

def concat_da2d(selection, filename_cols, directory):
    da_out = None
    sites = []
    for index, row in selection.iterrows():
        channels = []
        for f in filename_cols:
            path = directory / row[f]
            #print(str(path))
            img = AICSImage(path)
            #channels.append(img.data)
            channels.append(img.get_image_dask_data())
            #print(img.get_image_dask_data().shape)
        site = da.concatenate(channels, axis=2)
        sites.append(site)

    da_out = da.concatenate(sites, axis=0)
    return da_out



In [ ]:
import dask.array as da

dask_arrays = {}
index_map = {}

def channels2da(directory, filenames):
    return da.stack([AICSImage(directory / f).get_image_dask_data() for f in filenames])
    
for (plate, well, site), group in grouped3d.iterrows():
    # Assuming the file paths for the Z-slices are in group['file_paths']
    dask_array = da.stack([channels2da(orig, fileset) for fileset in group[filename_cols]])
    
    # Store the Dask array and its corresponding index
    dask_arrays[(well, site)] = dask_array
    index_map[(well, site)] = len(dask_arrays) - 1  # Map (well, site) to Dask array index


In [ ]:
A_out = concat_da2d(df2d, filename_cols, orig)
print(A_out.shape)

In [ ]:
from qtpy.QtWidgets import QLabel, QWidget, QVBoxLayout

viewer = napari.view_image(
        A_out,
        channel_axis=2,
        name=wavelengths,
        colormap=colormap,
        #contrast_limits=[[200, 4095], [500, 4095], [200, 4095]],
        )

# Create a custom widget to display the stack index
class StackIndexWidget(QWidget):
    def __init__(self):
        super().__init__()
        self.label = QLabel("Current stack index: 0")
        #self.label_file = QLabel("Filename w1")
        self.label_plate = QLabel("Plate: N/A")
        self.label_well = QLabel("Well: N/A")
        self.label_site = QLabel("Site: N/A")
        layout = QVBoxLayout()
        layout.addWidget(self.label)
        #layout.addWidget(self.label_file)
        layout.addWidget(self.label_plate)
        layout.addWidget(self.label_well)
        layout.addWidget(self.label_site)
        self.setLayout(layout)

    def update_index(self, index):
        self.label.setText(f"Current stack index: {index}")
        #self.label_file.setText(f"Filename w1: {df.loc[index, 'FileName_w1']}")
        self.label_plate.setText(f"Plate: {df.loc[index, PLATE]}")
        self.label_well.setText(f"Well: {df.loc[index, WELL]}")
        self.label_site.setText(f"Site: {df.loc[index, SITE]}")

# Instantiate the custom widget
stack_index_widget = StackIndexWidget()

# Add the widget to Napari as a dock widget
viewer.window.add_dock_widget(stack_index_widget, name="Stack Index")

# Callback function to update the widget with the current stack index
def on_index_change(event):
    current_index = viewer.dims.current_step[0]
    stack_index_widget.update_index(current_index)

# Connect the callback to the dims event
viewer.dims.events.current_step.connect(on_index_change)
